# Train an Object Detection Model with GeoAI

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/train_object_detection_model.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [ ]:
# %pip install geoai-py

In [8]:
import ee
import geemap

# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize(project='ee-mhqadi')




In [9]:
# Define your area of interest (AOI) - Modify this to your desired region
aoi = ee.Geometry.Rectangle([35.8, 33.84, 36.5, 31.4])  # Example: Adjust as needed

# Define time range for Sentinel-2
start_date = '2025-01-01'
end_date = '2025-01-31'

# Load Sentinel-2 collection, filter by date and cloud cover
s2_collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(aoi)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
    .median()  # Take median to reduce cloud effects
)

# Select 10m resolution bands
bands_10m = ["B2", "B3", "B4", "B8"]
bands_20m = ["B5", "B6", "B7", "B8A", "B11", "B12"]

# Resample 20m bands to 10m
s2_resampled = s2_collection.select(bands_20m).resample('bilinear').reproject(crs='EPSG:4326', scale=10)

# Merge all bands into one image
s2_final = s2_collection.select(bands_10m).addBands(s2_resampled)

# Export to Google Drive as a single GeoTIFF
task = ee.batch.Export.image.toDrive(
    image=s2_final.clip(aoi),
    description="Sentinel2_10m_AllBands",
    folder="GEE_Downloads",
    fileNamePrefix="Sentinel2_AllBands_10m",
    region=aoi,
    scale=10,
    crs="EPSG:4326",
    maxPixels=1e13
)

# Start the export task
task.start()

print("Export started. Check Google Drive for the downloaded TIFF.")

Export started. Check Google Drive for the downloaded TIFF.


In [14]:
from google.colab import drive
drive.mount('/mydrive')

Mounted at /mydrive


In [12]:

import rasterio
import numpy as np
import matplotlib.pyplot as plt


## Import libraries

## Download sample data

In [16]:
train_raster_url = ("/content/Sentinel2_Specified_Area_10mBands.tif"
)
train_vector_url = "/content/hafir.json"
test_raster_url = (
    "/content/Sentinel2_Amman_10mBands.tif"
)

In [ ]:
%pip install geoai-py
import geoai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
 

In [17]:
train_raster_path = geoai.download_file(train_raster_url)
train_vector_path = geoai.download_file(train_vector_url)
test_raster_path = geoai.download_file(test_raster_url)

NameError: name 'geoai' is not defined

## Visualize sample data

In [ ]:
geoai.view_vector_interactive(train_vector_path, tiles=train_raster_url)

In [ ]:
geoai.view_raster(test_raster_url)

## Create training data

In [ ]:
out_folder = "output"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_vector_path,
    tile_size=512,
    stride=256,
    buffer_radius=0,
)

## Train object detection model

In [ ]:
geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models",
    num_channels=4,
    pretrained=True,
    batch_size=4,
    num_epochs=10,
    learning_rate=0.005,
    val_split=0.2,
)

## Run inference

In [ ]:
masks_path = "naip_test_prediction.tif"
model_path = f"{out_folder}/models/best_model.pth"

In [ ]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=512,
    overlap=256,
    confidence_threshold=0.5,
    batch_size=4,
    num_channels=4,
)

## Vectorize masks

In [ ]:
output_path = "naip_test_prediction.geojson"
gdf = geoai.orthogonalize(masks_path, output_path, epsilon=2)

## Visualize results

In [ ]:
geoai.view_vector_interactive(output_path, tiles=test_raster_url)

In [ ]:
geoai.create_split_map(
    left_layer=output_path,
    right_layer=test_raster_url,
    left_args={"style": {"color": "red", "fillOpacity": 0.2}},
    basemap=test_raster_url,
)

![image](https://github.com/user-attachments/assets/8dfcc69e-7a6c-408a-9fae-10b81b7d85dc)